In [1]:
import spacy
from tqdm.auto import tqdm
from spacy.tokens import DocBin
from ml_datasets import imdb
train_data, valid_data = imdb()
# pipeline только из tok2vec
nlp = spacy.load("en_core_web_md", exclude=['tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner'])

In [2]:
nlp.pipe_names

['tok2vec']

In [3]:
def make_docs(data):
    """
    this will take a list of texts and labels
    and transform them in spacy documents
    data: list(tuple(text, label))
    returns: List(spacy.Doc.doc)
    """
    docs = []
    # nlp.pipe([texts]) is way faster than running
    # nlp(text) for each text
    # as_tuples allows us to pass in a tuple,
    # the first one is treated as text
    # the second one will get returned as it is.
#     a = tqdm(nlp.pipe(data, as_tuples=True), total = len(data))
    for doc, label in tqdm(nlp.pipe(data, as_tuples=True), total = len(data)):
        if label == 'neg':
            doc.cats["positive"] = 0
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return docs

In [4]:
# это для примера, можно взять больше количество текстов
num_texts = 1500
train_docs = make_docs(train_data[:num_texts])

  0%|          | 0/1500 [00:00<?, ?it/s]

In [5]:
# then we save it in a binary file to disc
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("train.spacy")
# repeat for validation data
valid_docs = make_docs(valid_data[:num_texts])
doc_bin = DocBin(docs=valid_docs)
doc_bin.to_disk("valid.spacy")

  0%|          | 0/1500 [00:00<?, ?it/s]

https://spacy.io/usage/training#quickstart

In [6]:
# что здесь происходит: мы заполнили base_config, а эта команда на его основе прописывает config.cfg
! python -m spacy init fill-config base_config_all.cfg config_all.cfg

✔ Auto-filled config with all values
✔ Saved config
config_all.cfg
You can now add your data and train your pipeline:
python -m spacy train config_all.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


max_steps = 4000

In [8]:
! python -m spacy train config_all.cfg --output ./output

ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  SCORE 
---  ------  ------------  ------
  0       0          0.00    0.00
  0     200          0.00    0.00
  0     400          0.00    0.00
  0     600          0.00    0.00
  0     800          0.00    0.00
  0    1000          0.00    0.00
  0    1200          0.00    0.00
  0    1400          0.00    0.00
  1    1600          0.00    0.00
✔ Saved pipeline to output directory
output\model-last
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec']
ℹ Initial learn rate: 0.001
E    # 

In [9]:
nlp = spacy.load("output/model-best")

In [10]:
nlp('nice happy littens').vector

array([ 0.53132516, -0.7366123 , -0.4063084 , -0.08459791,  1.0954684 ,
        1.0159502 , -1.2712227 ,  0.68313473, -4.060801  ,  1.6669806 ,
       -0.12979746, -2.5603354 ,  0.31405786,  2.3830335 ,  2.3257105 ,
        0.3503945 ,  0.30127546, -1.1691624 ,  2.885957  ,  1.278132  ,
        0.8772423 ,  2.482701  ,  0.5558037 ,  0.44140545, -1.8200301 ,
       -0.65617675, -1.9190811 ,  0.68270665, -1.5138842 , -0.18671973,
        0.71070117, -0.04105099,  0.47409734,  0.37996277,  1.0666336 ,
       -1.010496  , -0.9148996 ,  0.34500232, -0.9718332 ,  2.1734579 ,
       -1.1052717 ,  0.33938706,  0.336931  , -0.10465869, -2.721653  ,
       -0.34240374,  1.3156608 , -1.0238277 ,  0.8156789 ,  0.62916064,
        0.56528825,  2.6818469 ,  0.87224483,  2.3990521 , -0.74007815,
        0.6015846 , -1.958942  , -0.7118247 , -0.23374371,  0.4998633 ,
        1.0785441 ,  1.0384414 , -1.2045697 , -1.0150666 , -1.7387968 ,
       -0.16804051,  0.04633713,  0.13312156, -2.135768  ,  0.49

Косинусное расстояние – это мера похожести двух векторов.

Косинусное расстояние между векторами `u` и `v`:
$1 - \dfrac{u \cdot v}{\|u\|_2 \|v\|_2}$

Косинусное расстояние может быть от 0 до 2, где 0 означает, что векторы идентичны, а 2 означает, что они противоположны. Если векторы ортогональны (то есть угол между ними составляет 90 градусов), косинусное расстояние будет равно 1.

In [11]:
from scipy.spatial.distance import cosine

In [12]:
cosine(nlp('great').vector, nlp('nice').vector)

1.1255535334348679

Ничего не получилось, переходим к Word2Vec